In [340]:
import numpy as np
import pandas as pd

1. Читаем из файла:

In [341]:
df = pd.read_csv('flatest.csv', sep=';',  encoding='cp1251', na_values='?')

2. Приводим столбец Цена к числовому типу:

In [342]:
df['Цена,руб'] = df['Цена,руб'].replace(to_replace=r'\s+', value='',regex=True).astype('int64')

3. Удаляем дубликаты квартир:

In [343]:
df = df.sort_values(by='Цена,руб')
df = df.drop_duplicates(subset=['Улица', '№ дома', 'Дом', 'Площадь'], keep = 'first')
df = df.sort_index()

4. Разбиваем столбец Дом на следующие столбцы:

In [344]:
df.insert(6, 'Этаж', '0') #этаж на котором квартира
df.insert(7, 'Этажность', '0') #количество этажей в доме
df.insert(8, 'Первый этаж', '0' ) #да = 1, нет = 0
df.insert(9, 'Крайний этаж', '0') #первый или последний = 1,нет = 0
df.insert(10, 'Отношение', '0') #отношение этажа к количеству этажей
df.insert(11, 'Тип дома', '0') #тип дома Б - блочный П - панельный М - монолит М-К - монолит-кирпич К - кирпич

In [345]:
df['Этаж'] = df['Дом'].str.split('/').str.get(0).replace('\\?', np.nan, regex=True)
print(df['Этаж'])

0       1
1       1
2       9
4       8
5       7
6      11
8       8
9       3
10      1
11      1
12      1
14      1
15      2
16      5
17      3
18      5
19      1
22      2
23      5
24     11
25      7
27      1
28      5
31      4
32      2
33     13
34    NaN
Name: Этаж, dtype: object


In [346]:
df['Этажность'] = df['Дом'].str.split('/').str.get(1).str.split(' ').str.get(0).replace('\\?', np.nan, regex=True).fillna(0)
df['Этажность']

0      5
1      9
2      9
4      9
5      9
6     15
8      9
9      5
10     5
11    17
12     9
14     9
15    12
16     5
17     5
18     5
19     9
22     5
23     5
24    12
25     9
27     5
28     5
31     5
32     5
33     0
34    20
Name: Этажность, dtype: object

In [347]:
df['Первый этаж'] =  ~df['Дом'].str.split('/').str.get(0).replace('\\?', np.nan, regex=True).astype('float').replace(1, 0).astype('bool')
df['Первый этаж']

0      True
1      True
2     False
4     False
5     False
6     False
8     False
9     False
10     True
11     True
12     True
14     True
15    False
16    False
17    False
18    False
19     True
22    False
23    False
24    False
25    False
27     True
28    False
31    False
32    False
33    False
34    False
Name: Первый этаж, dtype: bool

In [348]:
df['Крайний этаж'] =  df['Дом'].str.split('/').str.get(0).replace('\\?', np.nan, regex=True).astype('float').fillna(0)
df['Крайний этаж'] = df['Крайний этаж'].fillna(0) - df['Этажность'].fillna(0)#= ~(df['Крайний этаж'] - df['Этажность']).astype('bool')
df['Крайний этаж'] #= df['Крайний этаж'] | df['Первый этаж']

TypeError: unsupported operand type(s) for -: 'float' and 'str'

In [349]:
df['Отношение'] = (df['Этаж'] /df['Этажность']).astype('float').round(3) 

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [350]:
df['Тип дома'] = df['Дом'].str.split(' ').str.get(1)

5. Анализируем дома в зависимости от типа:

In [351]:
df.groupby('Тип дома').size()

Тип дома
?     4
Б     8
К     4
П    11
dtype: int64

In [352]:
df.to_csv('flatest_.csv', sep=';',  encoding='cp1251')